# Divide and Conquer

* DC1: Fast Integer Multiplication
* DC2: Linear-time Median
* DC3: Solving Recurrences
* DC4, DC5: Fast Polynomial Multiplication using Fast Fourier Transform (FFT)

#### What is Divide and Conquer?
* Divide and Conquer algorithms solve problems in three steps:
    1. Divide: Break the given problem into subproblems of same type.
    2. Conquer: Recursively solve these subproblems
    3. Combine: Appropriately combine the answers
* Examples: binary search, quicksort, mergesort

#### Divide and Conquer vs. Dynamic Programming
* Divide and Conquer should be used when same subproblems are not evaluated many times. Otherwise Dynamic Programming or Memoization should be used.

[Resource 1](https://classroom.udacity.com/courses/ud401)
[Resource 2](https://www.geeksforgeeks.org/divide-and-conquer-introduction/)

---
# DC1

---
### Fast Integer Multiplication
* Problem: Given 2 n-bit integers x and y (when n is large). Compute z=xy efficiently.

#### Naive approach 
* Run time O(n^2) for naive approach

#### Simple divide and conquer approach
* Divide x and y into halves: `x=2^(n/2)*xL + xR` and `y=2^(n/2)*yL + yR`
* Run time: `T(n)=4T(n/2)+O(n)=O(n^2)`
* 4 multiplcations => not better than naive approach

#### Improved divide and conquer approach using Gauss's idea
* Use Gauss's idea to reduce 4 multiplcations into 3 multiplcation when computing (a+bi)(c+di)
* That is, `xy = (2^(n/2)*xL + xR)(2^(n/2)*yL + yR) = 2^n*xL*yL + 2^(n/2)*(xL*yR + xR*yL) + xR*yR`. Then use the trick `xL*yR + xR*yL =(xL+xR)(yL+yR) - xL*yL - yR*yR`
* Run time: `T(n)=3T(n/2)+O(n)~O(n^1.59)`
* Use 3 multiplcations instead => better performance

[Reference](https://www.geeksforgeeks.org/divide-and-conquer-set-2-karatsuba-algorithm-for-fast-multiplication/)

In [85]:
# int to bit
print(str(bin(12))[2:], str(bin(14))[2:])

# bit to integer
print(int('1100',2), int('1110',2))

print(12*14, bin(12*14), bin(0b1100*0b1110))

1100 1110
12 14
168 0b10101000 0b10101000


In [2]:
def fastMultiply1(x,y):
#     print('check1:', x,y)

    if len(str(x)) == 1 or len(str(y)) == 1:
        return x*y
    else:
        n = max(len(str(x)),len(str(y)))
        nby2 = n // 2
    
        xL = x // 10**(nby2)
        xR = x % 10**(nby2)
        yL = y // 10**(nby2)
        yR = y % 10**(nby2)
#         print('check2:', xL,xR,yL,yR)

        # xy = (2^(n/2)*xL + xR)(2^(n/2)*yL + yR) = 2^n*xL*yL + 2^(n/2)*(xL*yR + xR*yL) + xR*yR
        # Then use the trick xL*yR + xR*yL =(xL+xR)(yL+yR) - xL*yL - yR*yR
        A = fastMultiply1(xL,yL)
        B = fastMultiply1(xR,yR)
        C = fastMultiply1(xL+xR,yL+yR)
        return (2**n)*A + (2**(n//2))*(C-A-B) + B
    
fastMultiply1(1100,1110)


168

In [3]:
# something wrong

def fastMultiply2(x,y):
    '''
    Input 2 n-bit numbers x,y in string format
    COmpute z=xy
    '''
#     print('check1:', x,y)
    
    if len(x) == 1 or len(y) == 1:
        return int(x,2)*int(y,2)
    else:
        n = max(len(x),len(y))
        nby2 = n // 2
    
        xL = x[:nby2]
        xR = x[nby2:]
        yL = y[:nby2]
        yR = y[nby2:]
        xLaddxR = str(bin(int(xL,2)+int(xR,2)))[2:]
        yLaddyR = str(bin(int(yL,2)+int(yR,2)))[2:]
#         print('check2:', xL,xR,yL,yR)
        
        # xy = (2^(n/2)*xL + xR)(2^(n/2)*yL + yR) = 2^n*xL*yL + 2^(n/2)*(xL*yR + xR*yL) + xR*yR
        # Then use the trick xL*yR + xR*yL =(xL+xR)(yL+yR) - xL*yL - yR*yR
        A = fastMultiply2(xL,yL)
        B = fastMultiply2(xR,yR)
        C = fastMultiply2(xLaddxR,yLaddyR)
        
        return (2**n)*A + (2**nby2)*(C-A-B) + B
    
fastMultiply2('1100','1110')

160

---
# DC2

---
### Linear-time Median

#### Problem
* Problem: Given an unsorted list A=[a1,...,an] of n numbers. Find the median of A (for odd n=2l+1, median is the (l+1)st smallest).
* More general problem: Given an unsorted list A=[a1,...,an] of n numbers and k where 1<=k<=n. Find the kth smallest of A.

#### Solutions
* Easy Algorithm: Sort A and then output the kth element. MergeSort run time O(nlogn).
* Divide and Conquer Approach using Random pivot. Expected run time O(n) time.
* Divide and Conquer Approach using Recursive pivot. Worst run time O(n) time.

#### DC Approach using Recusive Pivot - Idea
* QuickSort
    * Choose a pivot p
    * Partition A into A< p, A=p and A> p
    * Recursively sort A< p and A> p
    * The challenge is to choose a good pivot (eg. median)
* Good pivot
    * A pivot p is good if len(A<p)<=3n/4 and len(A>p)<=3n/4
    * The goal is to find a good pivot p in O(n) time, i.e. `T(n)=T(3n/4)+O(n)=O(n)`
    * Radnom pivot has expected run time O(n) becasue a ranodm pivot has 0.5 probability being a good pivot, but we want to guarantee that the worst run time is also O(n)
    * To find a good pivot, we can divide A into n/5 group, take medians of subgroup as a reppresentative sample, and then recursively find median of the sample as our pivot. It is a good pivot becasue `T(n)=T(3n/4)+T(n/5)+O(n)=O(n)` because 3/4+1/5<1
    
####  DC Approach using Recusive Pivot - Pseudocode
1. Break A into n/5 group, `G_1,G_2,...,G_n/5`
2. For i=1 to n/5, sort G_i and let `m_i=median(G_i)`
3. Let `S={m(1,...,m_n/5}`
4. `p = FastSelect(S,n/10)` i.e. the median of S
5. Partition A into `A<p, A=p and A>p`
6. Recurse on A< p or A> p or output p
    * If `k<=len(A<p)` then return `FastSelect(A<p, k)`
    * If `k>len(A<p)+len(A=p)` then return `FastSelect(A>p, k-len(A<p)-len(A=p))`
    * Else output p
    
[Reference 1](https://www.geeksforgeeks.org/kth-smallestlargest-element-unsorted-array-set-3-worst-case-linear-time/)
[Reference 2](http://www.ardendertat.com/2011/10/27/programming-interview-questions-10-kth-largest-element-in-array/)

In [36]:
def findKMin1(A,k):
    '''    
    Input: unsorted array A of size n & integer k where 1<=k<=n
    Return kth smallest of A by sorting the array first 
    Expected run time: O(nlogn)
    '''
    A = sorted(A)
    return A[k]
    
findKMin1([4,5,7,1,2,8,3,3,6,9],3)

3

In [39]:
from random import randrange

def findKMin2(A,k,start=0,end=None):
    '''    
    Input: unsorted array A of size n & integer k where 1<=k<=n
    Return kth smallest of A using random pivot (similar to quick sort)
    Expected run time: O(n)
    assumption: Input will only contain unique elements
    '''
    if k > len(A):
        raise Exception("k should be less than the length of the input array")
        
    if not end:
        end = len(A)-1

    pivot_idx = randrange(start,end)  # random pivot
    pivot = A[pivot_idx]
    new_pivot_idx = partition2(A,start,end,pivot_idx)
    
    if new_pivot_idx+1 == k:
        return pivot
    elif new_pivot_idx+1 > k:
        return findKMin2(A,k,start,new_pivot_idx)
    else:
        return findKMin2(A,k,new_pivot_idx,end)

def partition2(A,start,end,pivot_idx):
    '''Helper function to partitions array in-place around the pivot value'''
    pivot = A[pivot_idx]
    A[end], A[pivot_idx] = A[pivot_idx], A[end]
    new_idx = start
    for i in range(start,end):
        if A[i] <= pivot:
            A[new_idx], A[i] = A[i], A[new_idx]
            new_idx += 1
    A[end], A[new_idx] = A[new_idx], A[end]
#     print('check', A, pivot)
    return new_idx

findKMin2([4,5,7,1,2,8,3,3,6,9],4)

3

In [72]:
def findKMin3(A, left, right, k):
    '''    
    Input: unsorted array A of size n & integer k where 1<=k<=n
    Return kth smallest of A using recursive pivot (median of medians)
    Worst run time: O(n)
    
    Idea is to divide A into subarray of length 5, 
    recursively find the kth smallest of the medians of subarrays a
    
    1. Break A into n/5 group, G_1,G_2,...,G_n/5
    2. For i=1 to n/5, sort G_i and let m_i=median(G_i)
    3. Let S={m(1,...,m_n/5}
    4. p = FastSelect(S,n/10) i.e. the median of S
    5. Partition A into A<p, A=p and A>p
    6. Recurse on A< p or A> p or output p
            If k<=len(A<p) then return FastSelect(A<p, k)
            If k>len(A<p)+len(A=p) then return FastSelect(A>p, k-len(A<p)-len(A=p))
            Else output p
    '''
    # base case
    length = right-left+1
    if not 1<=k<=length:
        return
    if length<=5:
        return sorted(A[left:right+1])[k-1]
    
    # divide A into subarrays of length 5, and recursively find medians of subarrays
    numMedians = length//5
    medians = [findKMin3(A, left+5*i, left+5*(i+1)-1, 3) for i in range(numMedians)]
    print('check medians', medians)
    
    # recursively find median of medians as pivot
    pivot = findKMin3(medians, 0, len(medians)-1, len(medians)//2+1)
    print('check pivot', pivot)
    
    # partition A around pivot
    pivotIndex = partition3(A, left, right, pivot)
    print('check pivot index', pivotIndex)
    
    # recurse on A<=p and A>p
    rank = pivotIndex-left+1
    if k<=rank:
        return findKMin3(A, left, pivotIndex, k)
    else:
        return findKMin3(A, pivotIndex+1, right, k-rank)
    
    
def partition3(A, left, right, pivot):
    '''Helper function to partitions array in-place around the pivot value'''
    swapIndex=left
    for i in range(left, right+1):
        if A[i]<pivot:
            A[i], A[swapIndex] = A[swapIndex], A[i]
            swapIndex+=1
    return swapIndex-1


findKMin3([4,5,7,1,2,8,3,3,6,10,11],0,10,4)

check medians [4, 6]
check pivot 6
check pivot index 5
check medians [3]
check pivot 3
check pivot index 1


3

---
# DC4, DC5

---
### Polynomial Multiplication using Fast Fourier Transform (FFT)

#### Problem
* Input: `a=(a0,a1,...,an-1)` and `b=(b0,b1,...,bn-1)`
* Output: convolution `c=a*b=c0,c1,...,c2n-1)` where `ck=a0*bk+a1*bk-1+...+ak*b0`
* In the other words, given two polynomicals `A(x)=a0+a1*x+...+an-1*x^(n-1)` and `B(x)=b0+b1*x+...+bn-1*x^(n-1)`. Compute `C(x)=A(x)B(x)=c0+c1*x+...+c2n-2*x^(2n-2)` where `ck=a0*bk+a1*bk-1+...+ak*b0`

#### Naive Approach to solve polynomial multiplication
* O(k) time for ck => Run time O(n^2)
* Can we do it in O(nlogn) time?
    
#### FFT  (Divide and Conquer Algorithm)
* Use FFT to convert coefficients of A(x) and B(x) to values representation. Run time `T(n)=2T(n/2)+O(n) = O(nlogn)`
* Multiply polynomials in values representation: `C(xi)=A(xi)B(xi)`. Run time O(n).
* Use Invserse FFT to convert values representation to coeffieints of C(x). Run time O(nlogn).

[Reference](https://www.geeksforgeeks.org/fast-fourier-transformation-poynomial-multiplication/)

In [78]:
# Naive approach for polynomical multiplication
def polynomialMultiply(A, B):
    '''
    Run time O(mn)
    Space O(mn)
    '''
    m = len(A)
    n = len(B)
    prod = [0]*(m+n-1)
    for i in range(m):
        for j in range(n):
            prod[i+j] += A[i]*B[j] 
    return prod

A = [5,0,10,6] # A=5+10x^2+6x^3
B = [1,2,4]   # B=1+2x+4x^2
polynomialMultiply(A, B) #prod=5+10x+30x^2+26x^3+52x^4+24x^5


[0, 0, 0, 0, 0, 0]


[5, 10, 30, 26, 52, 24]

In [ ]:
# Polynomail multiplication using FFT
def FFT(A,B):
    '''
    Input A=(a0,...,an-1) and B=(b0,...,bn-1) coefficients of polynomials A(x) and B(x)
    Return C=(c0,...,c2n-2) coefficients of C(x)=A(x)B(x)
    Run time O(nlogn)
    
    Idea:
    (r0,...,r2n-1) = FFT(A, w2n)
    (s0,...,s2n-1) = FFT(B, w2n)
    for j=0 to 2n-1, tj = rj*sj
    (c0,...,c2n-1) = 1/2n * FFT(t, w2n^2n-1)
    '''
    return 

In [80]:
#https://gist.github.com/berenoguz/f8bd037a82a23737a560d695cc9d6a0efrom cmath import exp

from math import pi

class NthRootOfUnity:
    '''
    A simple class to simulate n-th root of unity
    This class is by no means complete and is implemented merely for FFT and FPM algorithms
    '''
    def __init__(self, n, k = 1):
        self.k = k
        self.n = n

    def __pow__(self, other):
        if type(other) is int:
            n = NthRootOfUnity(self.n, self.k * other)
            return n

    def __eq__(self, other):
        if other == 1:
            return abs(self.n) == abs(self.k)

    def __mul__(self, other):
        return exp(2*1j*pi*self.k/self.n)*other

    def __repr__(self):
        return str(self.n) + "-th root of unity to the " + str(self.k)

    @property
    def th(self):
        return abs(self.n // self.k)

def FFT(A, omega):
    '''
    The Fast Fourier Transform Algorithm
    Input: A, An array of integers of size n representing a polynomial
           omega, a root of unity
    Output: [A(omega), A(omega^2), ..., A(omega^(n-1))]
    Run time: O(n logn)
    '''
    if omega == 1:
        return [sum(A)]
    B = [[],[]]
    i = 0
    for a in A:
        B[i%2].append(a)
        i+=1
    o2 = omega**2
    C1 = FFT(B[0], o2)
    C2 = FFT(B[1], o2)
    C3 = [None]*omega.th
    for i in range(omega.th//2):
        C3[i] = C1[i] + omega**i * C2[i]
        C3[i+omega.th//2] = C1[i] - omega**i * C2[i]
    return C3

def FPM(A,B):
    '''
    The Fast Polynomial Multiplication Algorithm
    Input: A,B arrays of integers representing polynomials of lenth O(n)
    Output: Coefficient representation of AB
    Run time: O(n logn)
    '''
    n = 1<<(len(A)+len(B)-2).bit_length()
    o = NthRootOfUnity(n)
    AT = FFT(A, o)
    BT = FFT(B, o)
    C = [AT[i]*BT[i] for i in range(n)]
    nm = (len(A)+len(B)-1)
    D = [int((a/n).real) for a in FFT(C, o ** -1)]
    while True:
        if D[-1] != 0:
            return D
        else:
            del D[-1]

In [81]:
A = [5,0,10,6] # A=5+10x^2+6x^3
B = [1,2,4]   # B=1+2x+4x^2
FPM(A, B) #prod=5+10x+30x^2+26x^3+52x^4+24x^5

[5, 10, 30, 26, 52, 24]